In [723]:
# bibliotecas
import pandas as pd
import time
# funções 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# modelos
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

|             | Random Forest(est= 11, depth=27)   | Random Forest(est=100, depth=27)  | Linear Regression(fit_intercept, jobs=-1)   |
|-------------|------------|------------|------------|
| train_score     | 🟨0.9557926155884157    | 🟩0.9650129200912126    | 🟥0.6463556388680558    |
| test_score     | 🟨0.8334794802644814    |🟩0.8408169218841756    | 🟥0.5734454636073181    |
| error($)  | 🟨3931.966016272742    | 🟩3873.6263369273256    | 🟥6866.667619648236    |
| time(s)     | 🟨8.193240880966187    | 🟥79.8025176525116    | 🟩0.4817066192626953    |

# Random Forest
legenda: treino, teste, erro, tempo<br><br>
n_estimators=11, max_depth=27 -> 0.9557926155884157 0.8334794802644814 3931.966016272742 8.193240880966187<br> <br>
n_estimators=100, max_depth=27 -> 0.9650129200912126 0.8408169218841756 3873.6263369273256 79.8025176525116<br>
# Linear Regression
legenda: treino, teste, erro<br><br>
fit_intercept=True, n_jobs=-1 -> 0.6463556388680558 0.5734454636073181 6866.667619648236<br>

### Célula usada para encontrar os melhores hiperparâmetros da Random Forest

```best_error = float('inf')
best_est = 0
best_depth = 0

for est in range(10, 101, 5):
    for depth in range(20, 31):
        model = RandomForestRegressor(random_state=1506, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        predictions_test = model.predict(features_test)
        error = mean_squared_error(target_test, predictions_test)**0.5
        print(f"Validação de REQM para n_estimators={est}, depth={depth} é {error}")
        if error < best_error:
            best_error = error
            best_est = est
            best_depth = depth

print(f"Melhor REQM de teste: {best_error} com n_estimators={best_est} e max_depth={best_depth}")
```

# Melhor modelo

In [724]:
# carregando dataset
car_data = pd.read_csv('datasets/car_data.csv')

In [725]:
# retirando colunas que não serão utilizadas
car_data = car_data.drop(['date_posted', 'days_listed', 'year_posted', 'car_age'] , axis=1)

In [726]:
# transformando a coluna condition em numérica colocando "pesos" para cada condição
condition_mapping = {
    "new": 5,
    "like new": 4,
    "excellent": 3,
    "good": 2,
    "fair": 1,
    "salvage": 0
}

# atribuindo os valores numéricos ao dataset
car_data['condition'] = car_data['condition'].map(condition_mapping)

In [727]:
# separando features e target
features = car_data.drop('price', axis=1)
target = car_data['price']

In [728]:
# aplicando One-Hot Encoding nas variáveis categóricas restantes
features_encoded = pd.get_dummies(features, columns=['model', 'fuel', 'transmission', 'type', 'paint_color', 'brand'])

In [729]:
# separando dados de treino e teste
features_train, features_test, target_train, target_test = train_test_split(features_encoded, target, test_size=0.2)

In [730]:
model = RandomForestRegressor(random_state=1506, n_estimators=11, max_depth=27, min_samples_leaf=1)

In [731]:
start = time.time()
model.fit(features_train, target_train)
end = time.time()
tempo = end - start
tempo

8.289342880249023

In [732]:
print(model.score(features_train, target_train),
model.score(features_test, target_test))

0.9574493259906592 0.7330763826662909


In [733]:
predictions = model.predict(features_test)

In [734]:
rmse = mean_squared_error(target_test, predictions)**0.5
rmse

5376.266131474918